In [4]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms, utils
import time
import copy
from torch.autograd import Variable

# Process Data

In [5]:
# Data Reader Class

class DataReader(Dataset):
    def __init__(self):
        
        train_data = pd.read_csv("train.csv", header=None)[1:]
        
        self.train_x = np.asarray(self.data_info.iloc[:, 0:-1])
        self.train_y = np.asarray(self.data_info.iloc[:, -1])
        print("Read %d train samples" % len(self.train_y))

        test_data = pd.read_csv("test.csv", header=None)[1:]
        
        self.test_x = np.asarray(self.data_info.iloc[:, 0:-1])
        self.test_y = np.asarray(self.data_info.iloc[:, -1])

        print("Read %d test samples" % len(self.test_y))

        # get meta
        self.num_classes = 2
        self.input_size = self.train_x.shape[1]
        self.indexes = list(range(len(self.train_y)))
        self.train_size = len(self.train_y)
        
    def input_size(self):
        return self.input_size

    def init(self, batch_size):
        # shuffle
        self.batch_size = batch_size
        np.random.shuffle(self.indexes)
        return int(math.ceil(self.train_size / float(batch_size)))

    def get_batch(self, i):
        selected_idx = self.indexes[i*self.batch_size : (i+1)*self.batch_size]
        return self.train_x[selected_idx, :], self.train_y[selected_idx]

# Building Model

In [6]:
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.non_linear = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        var_x = Variable(torch.from_numpy(x).float())
        logitis = self.fc2(self.non_linear(self.fc1(var_x)))
        return nn.functional.log_softmax(logitis, dim=1)

# Evaluating Model

In [7]:
def eval(data_x, data_y, model, loss_func, name):
    model.eval()
    log_py = model(data_x)
    y = Variable(torch.from_numpy(data_y).long())
    l = loss_func(log_py, y).item()
    
    pred = np.argmax(log_py.data.numpy(), axis=1)
    acc = np.mean(pred == data_y)
    print("%s loss %f and acc %f " % (name, l, acc))
    return l, acc

# Training

In [8]:
NUM_EPOCH = 20
BATCH_SIZE = 32
LR = 0.1
MOMENTUM = 0.0
L2_DECAY = 0.0

data_loader = DataReader()
model = FNN(data_loader.input_size(), hidden_size=300, output_size=2)
nll_loss = nn.NLLLoss()
op = torch.optim.SGD(model.parameters(), lr=LR, 
                     momentum=MOMENTUM, weight_decay=L2_DECAY)

train_metric, test_metric = [], []
for i in range(NUM_EPOCH):
    print("Epoch %d" % i)
    num_batches = data_loader.init(batch_size=BATCH_SIZE)
    model.train()
    for b in range(num_batches):
        x, y = data_loader.get_batch(b)
        pred_y = model(x)
        y = Variable(torch.from_numpy(y).long())
        loss = nll_loss(pred_y, y)
        op.zero_grad()
        loss.backward()
        op.step()

    train_metric.append(eval(data_loader.train_x, data_loader.train_y, model, nll_loss, "TRAIN"))
    dev_metric.append(eval(data_loader.dev_x, data_loader.dev_y, model, nll_loss, "TEST"))

IOError: File train.csv does not exist